In [ ]:
# Here i haven't used chatgpt, i my self did this to learn new concepts, bcz if we keep on writing prompts you will loose your ability to think
# No thinking no new ideas or concepts
# use AI only when needed 

In [ ]:
!pip install pypdf sentence-transformers 

In [ ]:
!pip install faiss-cpu


In [35]:
import os
import numpy as np
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
import pickle


In [36]:
# The first step is to extract the information, Here iam extracting text from pdf
# 
def ext_text(path):
    data = PdfReader(path)
    df=""
    for x in data.pages:
        x_text=x.extract_text()
        if x_text:
            df += x_text +"\n"

    return df        



In [ ]:
path="/data/LM/rag_project/notes.pdf"
data = ext_text(path)
data

In [38]:
# Here we will chunk the text with over lap
def chunk_text(text, chunk_size=500,overlap=100):
    chunks=[]
    start=0 
    while start < len(text):
        end =  start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size-overlap
    return chunks

In [ ]:
chunks=chunk_text(data)
print(len(chunks),chunks)

In [ ]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
## Load Embedding Model
## emb =  SentenceTransformer("BAAI/bge-large-en-v1.5",device="cuda:6")
emb = SentenceTransformer("all-MiniLM-L6-v2", device="cuda:6")   

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 571.38it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [42]:
# Here we will create embeddings using the above model
embeddings = emb.encode(chunks,batch_size=10,convert_to_numpy=True,
show_progress_bar=True)
# here we convert each chunk into vector

Batches: 100%|██████████| 24/24 [00:00<00:00, 59.46it/s]


In [43]:
embeddings

array([[ 0.00338129, -0.09022815,  0.08497572, ..., -0.02394508,
        -0.08998082,  0.04077885],
       [-0.02748079, -0.02066522,  0.01795669, ..., -0.01886765,
        -0.01819193,  0.01213914],
       [ 0.0769441 , -0.06430613, -0.05270697, ...,  0.01947873,
        -0.0101842 , -0.04567917],
       ...,
       [-0.08240927,  0.00592057,  0.03266907, ...,  0.00999525,
         0.06170399, -0.02119865],
       [ 0.00646808,  0.04383314, -0.00831901, ..., -0.03141473,
         0.04569596, -0.02072829],
       [ 0.05067067, -0.03856865,  0.06469955, ..., -0.05472222,
        -0.05902956, -0.03183942]], dtype=float32)

In [44]:
# Here each chunk is 384 number, each chunk is one point in High dimensional space
# Faiss stores those points so we can ask later
embeddings[0].shape

(384,)

In [ ]:
#--------- Build FAISS Index --------####
# here we store the emeddings (vectors) in high dimensional space 

In [45]:
dimension = embeddings.shape[1]
# we create faiss index (nocompression with L2 distance)
index=faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [46]:
embeddings.shape

(231, 384)

In [ ]:
faiss.write_index(index, "/data/LM/rag_project/faiss_index.bin")
with open("/data/LM/rag_project/chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

Question Answering (RAG + LLM).

In [ ]:
index = faiss.read_index("/data/LM/rag_project/faiss_index.bin")

with open("/data/LM/rag_project/chunks.pkl", "rb") as f:
    chunks = pickle.load(f)


In [ ]:
## Load Small LLM
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)

model = model.to("cuda:6")   

Loading weights: 100%|██████████| 201/201 [00:00<00:00, 305.81it/s, Materializing param=model.norm.weight]                              


In [50]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rot

In [51]:
### Retrieval Function
def retrieve(q,k=3):
    q_emb=emb.encode([q],convert_to_numpy=True)
    D,I=index.search(q_emb,k)
    return [chunks[i] for i in I[0]]

##Question → embedding

##FAISS finds nearest chunks

##Returns top-k most relevant text pieces

##This is pure RAG retrieval.

In [52]:
## Generate Answer Function

def generate_answer(q):
    retrieve_chunk=retrieve(q)
    context="\n\n".join(retrieve_chunk)
    prompt=f""" You are a helpful Machine learning assistant.
    Answer ONLY using the provided CONTEXT.
    If answer not found, say "NOt found in Notes"

    Context:{context}
    Question:{q}
    Answer:
    """

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:6")

    output = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.3,
        do_sample=True
    )

    # Only decode newly generated tokens
    generated_tokens = output[0][inputs["input_ids"].shape[1]:]

    answer = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return answer.strip()

    

In [54]:
while True:
    question = input("\nAsk: ")

    if question.lower() in ["exit", "quit"]:
        break

    answer = generate_answer(question)
    print("\nAnswer:\n")
    print(answer)



Answer:

1. A machine learning system is a system that can learn from data without being explicitly programmed.
    2. Machine learning is a field of computer science that deals with the design, implementation, and application of algorithms that can learn from data.
    3. Machine learning systems can be used for a variety of tasks, including image recognition, speech recognition, natural language processing, and recommendation systems.
    4. Machine learning is often used in industries such as finance, healthcare, and marketing, where it can help improve decision-making processes and provide more accurate predictions.
    5. Machine learning is a rapidly growing field with many new techniques and approaches being developed every year.


In [ ]:
#THIS Is what we build in this experiment

# PDF → chunks → embeddings → FAISS
#                          ↓
# User Question → embed → search
#                          ↓
# Context + Question → LLM
#                          ↓
# Answer
